# Run Functions to Add Information to Projects

In [1]:
# ! pip install nltk

In [2]:
import numpy as np
import pandas as pd
from siuba import *

from calitp_data_analysis.sql import to_snakecase

from shared_utils import geography_utils

import dla_utils

import _script_utils
import _data_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/dask_geopandas/backends.py:13: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/l

In [3]:
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', None)

## Read in Data and function development

In [4]:
GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [5]:
### Read in data (First round)
# proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_(IIJA_Reporting)_ (3-9-23).xlsx", header=[3], skiprows=range(4,1784)))
# proj.drop(columns =['unnamed:_0'], axis=1, inplace=True)
# proj['summary_recipient_defined_text_field_1_value'] = proj['summary_recipient_defined_text_field_1_value'].fillna(value='None')


In [6]:
proj = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/IIJA_FMIS_AllProject_20230309_ToDLA.xlsx",))


In [7]:
proj.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value
862,44763,ER01,EMERGENCY REL 2022 SUPPLEMENT,39AQ001,0119000054S,IN MENDOCINO COUNTY AT VARIOUS LOCATIONS STORM DAMAGE REPAIRS; DEBRIS REMOVAL,45,Cong Dist 2,Active,IN MENDOCINO COUNTY AT VARIOUS LOCATIONS STORM DAMAGE REPAIRS; DEBRIS REMOVAL,17,Construction Engineering,150000.0,106200.0,S ER NONE


In [8]:
proj.county_code.value_counts()

37     284
23     124
999    105
19      96
67      84
59      81
71      81
73      77
65      71
1       66
85      61
7       59
83      58
87      56
45      53
81      47
17      46
29      43
79      42
53      41
13      38
97      37
95      36
41      35
89      35
47      34
107     34
77      34
55      32
61      30
115     26
99      25
57      25
9       17
75      14
105     14
39      13
91      12
113     11
15      11
111     11
33      10
69       9
103      9
93       8
43       8
11       7
31       7
63       6
21       6
27       5
109      5
5        3
35       3
101      3
51       2
25       1
Name: county_code, dtype: int64

In [9]:
# proj.congressional_district.

In [10]:
## move to _data_utils
# def add_new_codes(df):
#     new_codes = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FY21-22ProgramCodesAsOf5-25-2022.v2.xlsx"))
#     code_map = dict(new_codes[['iija_program_code', 'new_description']].values)
    
#     df['program_code_description'] = df.program_code.map(code_map)
#     proj['summary_recipient_defined_text_field_1_value'] = proj['summary_recipient_defined_text_field_1_value'].astype(str)
    
#     return df


In [11]:
proj2 = _data_utils.add_new_codes(proj)

In [12]:

    ## function that adds known agency name to df 
df = _script_utils.identify_agency(proj2, 'summary_recipient_defined_text_field_1_value')

In [13]:
# df.sample(3)

#### Breaking down Idenitfy Agency 

In [14]:
# #add project information for all projects
# def identify_agency1(df, identifier_col):
#     #projects wtih locodes
#     locode_proj = ((df[~df[identifier_col].str.contains(" ")]))
#     locode_proj = locode_proj>>filter(_[identifier_col]!='None')
    
#     locode_proj = (_data_utils.add_name_from_locode(locode_proj, 'summary_recipient_defined_text_field_1_value'))
    

#     #projects with no locodes
#     no_locode = ((df[df[identifier_col].str.contains(" ")]))
#     no_entry = df>>filter(_[identifier_col]=='None')
    
#     #concat no locodes and those with no entry
#     no_locode = pd.concat([no_locode, no_entry])
    
#     #add county codes:
#     county_base = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/Copy of County.xlsx", sheet_name='County', header=[1]))
#     county_base.drop(columns =['unnamed:_0', 'unnamed:_4'], axis=1, inplace=True)
#     county_base['county_description'] = county_base['county_description'] + " County"
    
#     #read in locode info
#     locodes = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"))
#     county_district = locodes>>group_by(_.district, _.county_name)>>count(_.county_name)>>select(_.district, _.county_name)>>filter(_.county_name!='Multi-County', _.district !=53)
    
#     # merge county information to add districts
#     county_info = (pd.merge(county_base, county_district, how='left', left_on= 'county_description', right_on = 'county_name'))
#     county_info.drop(columns =['county_name'], axis=1, inplace=True)
    
#     mapping1 = dict(county_info[['county_code', 'county_description']].values)
#     mapping2 = dict(county_info[['county_code', 'recipient_name']].values)
#     mapping3 = dict(county_info[['county_code', 'district']].values)
    
#     no_locode['county_description'] = no_locode.county_code.map(mapping1)
#     no_locode['recipient_name'] = no_locode.county_code.map(mapping2)
#     no_locode['district'] = no_locode.county_code.map(mapping3)
    
#     no_locode = no_locode.rename(columns = {"recipient_name":"implementing_agency", "county_description":"county_name"})

#     full_df = pd.concat([locode_proj, no_locode])
    
#     full_df.loc[full_df.county_name == "Statewide County", 'county_name'] = "Statewide"
    
#     full_df['implementing_agency'] = full_df['implementing_agency'].fillna(value='Unknown')
#     full_df['county_name'] = full_df['county_name'].fillna(value='Unknown')

    
#     return full_df


In [15]:
# #add project information for all projects

# locode_proj = ((proj2[~proj2['summary_recipient_defined_text_field_1_value'].str.contains(" ")]))
# locode_proj = locode_proj>>filter(_['summary_recipient_defined_text_field_1_value']!='None')
    
# locode_proj = (_data_utils.add_name_from_locode(locode_proj, 'summary_recipient_defined_text_field_1_value'))


In [16]:

# #projects with no locodes
# no_locode = ((proj2[proj2['summary_recipient_defined_text_field_1_value'].str.contains(" ")]))
# no_entry = proj2>>filter(_['summary_recipient_defined_text_field_1_value']=='None')
    
# #concat no locodes and those with no entry
# no_locode = pd.concat([no_locode, no_entry])
    

In [17]:
# len(no_locode)

In [18]:
# no_locode.sample()

In [19]:
# locode_proj.sample()

In [20]:
# #add county codes:
# county_base = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/Copy of County.xlsx", sheet_name='County', header=[1]))
# county_base.drop(columns =['unnamed:_0', 'unnamed:_4'], axis=1, inplace=True)
# county_base['county_description'] = county_base['county_description'] + " County"

# #read in locode info
# locodes = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"))
# county_district = locodes>>group_by(_.district, _.county_name)>>count(_.county_name)>>select(_.district, _.county_name)>>filter(_.county_name!='Multi-County', _.district !=53)

In [21]:
# # merge county information to add districts
# county_info = (pd.merge(county_base, county_district, how='left', left_on= 'county_description', right_on = 'county_name'))
# county_info.drop(columns =['county_name'], axis=1, inplace=True)
    
# mapping1 = dict(county_info[['county_code', 'county_description']].values)
# mapping2 = dict(county_info[['county_code', 'recipient_name']].values)
# mapping3 = dict(county_info[['county_code', 'district']].values)

In [22]:
   
# no_locode['county_description'] = no_locode.county_code.map(mapping1)
# no_locode['recipient_name'] = no_locode.county_code.map(mapping2)
# no_locode['district'] = no_locode.county_code.map(mapping3)
    

In [23]:
# no_locode = no_locode.rename(columns = {"recipient_name":"implementing_agency", "county_description":"county_name"})

# full_df = pd.concat([locode_proj, no_locode])

In [24]:
# no_locode.sample()

In [25]:
# full_df.loc[full_df.county_name == "Statewide County", 'county_name'] = "Statewide"


In [26]:
# (full_df
#  >>filter(_.summary_recipient_defined_text_field_1_value.str.contains('S '))).summary_recipient_defined_text_field_1_value.value_counts()

In [27]:
# (full_df
#  >>filter(_.summary_recipient_defined_text_field_1_value.str.contains('S '))
#  >>select(_.summary_recipient_defined_text_field_1_value, _.implementing_agency, _.county_code, _.county_name, _.project_description, _.project_title, _.program_code_description)
#  >>filter(_.summary_recipient_defined_text_field_1_value != ("S    ER NONE"))
# )


In [28]:
### hash out to test the find_alternative_name function on full_df.
# state_test = (full_df>>filter(_.county_name==('Statewide')) >>select(_.summary_recipient_defined_text_field_1_value, _.implementing_agency, _.county_code, _.county_name, _.project_description, _.project_title, _.program_code_description)
# )

#### Breakdown agency identify

In [29]:
# def read_in_locodes():
#     locodes = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"))
#     locodes['agency_name'] = locodes['agency_name'].str.upper()
#     locode_names = sorted(list(locodes['agency_name'].unique()), reverse=True)    
    
#     ## append list so that "Crosswalk" does not select "Ross"
#     locode_names.remove('ROSS')
#     locode_names.append("STATE PARKS")
#     locode_names.append("SACOG - Sacramento Area Council of Governments")
#     locode_names.append("SACOG")
#     locode_names.append("MTC")
#     locode_names.append("MTC - Metropolitan Transportation Commission")
#     locode_names.append("Statewide")
    
#     return locode_names

In [30]:
# def find_alternative_name(df, desc_col, new_col_name):
    
#     #read in locode names to get list to find in description column
#     locode_names = read_in_locodes()
    
#     ### https://stackoverflow.com/questions/68869434/create-an-pandas-column-if-a-string-from-a-list-matches-from-another-column
#     df[new_col_name] = df[desc_col].map(lambda s: next((name for name in locode_names if name in s), ""))
#     df[new_col_name] = df[new_col_name].str.title()
    
#     return df

In [31]:
## run the read_in_locodes, and find_alterntaive_names functions from the section Next Statewide

# test_title_df = find_alternative_name(full_df, 'project_description', "name_in_desc")

In [32]:
# test_title_df>>filter(_.name_in_desc == "")>>filter(_.county_name=="Statewide")

In [33]:
# ## changing the title function to include the new description col
# def add_new_title(df, first_col_method, second_col_type, third_col_name, alt_col_name, alt_in_proj_desc):
#     """
#     Function to add new title. 
#     Expected output example: "New Bike Lane in Eureka"
#     """    

    
#     def return_name(df):
        
#         if (df[third_col_name] == "California") & (df[alt_col_name] == "Statewide") & (df[alt_in_proj_desc] != ""):
#             return (df[first_col_method] + " " + df[second_col_type] +" in " + df[alt_in_proj_desc])
        
#         elif (df[third_col_name] == "California") & (df[alt_col_name] == "Statewide") & (df[alt_in_proj_desc] == ""):
#             return (df[first_col_method] + " " + df[second_col_type] +" " + df[alt_col_name])
                    
#         elif (df[third_col_name]== "Unknown"):
#             return (df[first_col_method] + " " + df[second_col_type] + " in " + df[alt_in_proj_desc])
        
#         elif (df[third_col_name] == "California"):
#             return (df[first_col_method] + " " + df[second_col_type] + " in " + df[alt_col_name])
        
#         elif (df[third_col_name] == "Metropolitan Transportation Commission"):
#             return (df[first_col_method] + " " + df[second_col_type] + " in the " + df[third_col_name] + " Region")
        
#         elif (df[third_col_name] == "Los Angeles County Metropolitan Transportation Authority"):
#             return (df[first_col_method] + " " + df[second_col_type] + " in the " + df[third_col_name] + " Region")
        
#         elif (df[third_col_name] == "Caltrans") & (df[alt_in_proj_desc] != ""):
#             return (df[first_col_method] + " " + df[second_col_type] + " in " + df[alt_in_proj_desc])
        
#         elif (df[third_col_name] == "Caltrans") & (df[alt_in_proj_desc] == ""):
#             return (df[third_col_name] + " " + df[first_col_method] + " " + df[second_col_type])
        
#         elif (df[third_col_name] != "California"):
#             return (df[first_col_method] + " " + df[second_col_type] + " in " + df[third_col_name])
        

#         return df

#     df['project_name_new'] = df.apply(return_name, axis = 1)
    
#     return df

In [34]:
# ## adding in the project title new types (in the broader add title funtion)

# full_df_copy = _script_utils.add_description(full_df, 'project_title')
    
# #remove project method column values so that the title function wont double count
# full_df_copy.loc[full_df_copy['project_type'] == 'Project', 'project_method'] = ""
# full_df_copy['project_type'] = full_df_copy['project_type'].replace('Project', np.NaN)
    
# #update for the projects not in the first round of descriptions
# full_df_copy =  _script_utils.add_description_4_no_match(full_df_copy, 'improvement_type_description')
# ## proj_unique_cat_title = update_no_matched(proj_unique_cat, "project_type", 'improvement_type_description', 'program_code_description')
    
# #fill nan values in 'Project_type' with values from 'project_type2' from add_description_4_no_match function
# full_df_copy['project_type'] = full_df_copy['project_type'].fillna(full_df_copy['project_type2'])

In [35]:
 # full_df_copy[['county_code', 'improvement_type',
 #     'implementing_agency_locode', 'district',
 #     'program_code_description', 'recipient_project_number',
 #          "project_method", "project_type", "implementing_agency", "county_name", "name_in_desc"]] = full_df_copy[['county_code', 'improvement_type',
 #                                                                     'implementing_agency_locode', 'district',
 #                                                                     'program_code_description', 'recipient_project_number',
 #                                                                    "project_method", "project_type", "implementing_agency", "county_name", "name_in_desc"]].astype(str)

In [36]:
# full_df_title = add_new_title(full_df_copy, "project_method", "project_type", "implementing_agency", "county_name", "name_in_desc")

In [37]:
# full_df_copy.sample()

In [38]:
# full_df>>filter(_.name_in_desc == "")>>filter(_.county_name=="Statewide")

In [39]:
# full_df_copy>>filter(_.name_in_desc=="")

In [40]:
# full_df_copy>>filter(_.county_name=="Statewide")>>select(_.summary_recipient_defined_text_field_1_value, _.implementing_agency, _.county_name, _.project_title, _.project_description, _.name_in_desc,
#                                                          _.project_method, _.project_type, _.project_type2, _.project_name_new)

In [41]:

    
    
# full_df['implementing_agency'] = full_df['implementing_agency'].fillna(value='Unknown')
# full_df['county_name'] = full_df['county_name'].fillna(value='Unknown')



##### Check unknowns

In [42]:
len(df>>filter(_.implementing_agency=='Unknown'))



1

In [43]:
len(df>>filter(_.county_name=='Unknown'))

1

In [44]:
df>>filter(_.implementing_agency=="Unknown")

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
706,44827,YR10,Recreational Trails Program,2023000,NaN,2023 Rec Trails Project: State Parks Incurred Administrative Expenses,999,Cong Dist 0,Active,"2023 Rec Trails Project: State Parks Incurred Administrative Expenses. State costs incurred administering the RTP such as staff time, meeting and travel costs and attendance at trail-related training sessions and conferences, Trail Advisory committee costs including meeting and travel costs for committee members, trail conference support, etc.",44,Other,274727.3,241760.0,nan,NaN,Unknown,NaN,Unknown,NaN,NaN


In [45]:
# df>>filter(_.project_title.str.contains("Administrative Expense"))

#### Reformat Columns

To change: 
* ~~District: 2 digit / integer~~
* ~~Congressional District number~~
* Add Progam Code Description and amount to a new project description

In [46]:
df = _data_utils.change_col_to_integer(df, "congressional_district")

In [47]:
df.sample()

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
995,44951,Y001,National Highway Performance Program (NHPP),5904156,0116000098L,VARIOUS LOCATIONS IN HUMBOLDT COUNTY BRIDGE PREVENTIVE MAINTENANCE: MINOR CONCRETE REPAIR ON 8 BRIDGES.,23,2,Active,VARIOUS LOCATIONS IN HUMBOLDT COUNTY BRIDGE PREVENTIVE MAINTENANCE: MINOR CONCRETE REPAIR ON 8 BRIDGES.,17,Construction Engineering,85274.98,75493.96,L5904NON-MPO,5904.0,Humboldt County,1.0,Humboldt County,Humboldt County Association of Governments,NON-MPO


In [48]:
## move to _data_utils
# def change_col_to_integer(df, col):
    
#     df[col] = df[col].str.split(' ').str[-1]
    
#     return df

In [49]:

# (df>>select(_.congressional_district, _.congressional_district_2)).sample(20)

In [50]:
proj2>>filter(_.summary_recipient_defined_text_field_1_value.str.contains("S "))

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value
0,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,Cong Dist 27,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",6,4R - Restoration & Rehabilitation,1983100.77,419227.50,S ER NONE
1,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,Cong Dist 27,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",17,Construction Engineering,793240.31,184329.22,S ER NONE
2,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,Cong Dist 32,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",6,4R - Restoration & Rehabilitation,16899.23,3572.50,S ER NONE
3,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,Cong Dist 32,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",17,Construction Engineering,6759.69,1570.78,S ER NONE
4,44579,ER01,Emergency Supplement Funding,39BD001,0719000177S,"IN LOS ANGELES COUNTY, IN EACH DIRECTION ON LA-39 IN THE UNINCORPORATED AREA EMERGENCY PROJECT - EMERGENCY OPENING",37,Cong Dist 27,Active,"ON STATE ROUTE: 39. IN LOS ANGELES COUNTY, IN EACH DIRECTION ON LA-39 IN THE UNINCORPORATED AREA EMERGENCY PROJECT - EMERGENCY OPENING",6,4R - Restoration & Rehabilitation,500000.00,61000.00,S ER NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2178,44993,ER01,Emergency Supplement Funding,31TV004,0218000119S,IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKM,63,Cong Dist 1,Active,ON STATE ROUTE: 70. IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKMENT,6,4R - Restoration & Rehabilitation,21004300.00,18412317.60,S NON-MPO
2179,44993,ER01,Emergency Supplement Funding,31TV004,0218000119S,IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKM,63,Cong Dist 1,Active,ON STATE ROUTE: 70. IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKMENT,17,Construction Engineering,2686000.00,2377900.00,S NON-MPO
2180,44993,ER01,Emergency Supplement Funding,31TV004,0218000119S,IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKM,63,Cong Dist 1,Active,ON STATE ROUTE: 70. IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTE

In [51]:
df>>filter(_.summary_recipient_defined_text_field_1_value.str.contains("S "))

,fmis_transaction_date,program_code,program_code_description,project_number,recipient_project_number,project_title,county_code,congressional_district,project_status_description,project_description,improvement_type,improvement_type_description,total_cost_amount,obligations_amount,summary_recipient_defined_text_field_1_value,implementing_agency_locode,implementing_agency,district,county_name,rtpa_name,mpo_name
0,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,27,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",6,4R - Restoration & Rehabilitation,1983100.77,419227.50,S ER NONE,NaN,California,7.0,Los Angeles County,NaN,NaN
1,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,27,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",17,Construction Engineering,793240.31,184329.22,S ER NONE,NaN,California,7.0,Los Angeles County,NaN,NaN
2,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,32,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",6,4R - Restoration & Rehabilitation,16899.23,3572.50,S ER NONE,NaN,California,7.0,Los Angeles County,NaN,NaN
3,44579,ER01,Emergency Supplement Funding,15D0001,0721000066S,"ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENI",37,32,Active,"ON STATE ROUTE: 39. ON LA-39 PM 18.0/30.9 & 39.5/44.4 IN ANGELES NATIONAL FOREST, IN THE UNINCORPORATED AREA OF COUNTY OF LOS ANGELES EMERGENCY PROJECT - EMERGENCY OPENING",17,Construction Engineering,6759.69,1570.78,S ER NONE,NaN,California,7.0,Los Angeles County,NaN,NaN
4,44579,ER01,Emergency Supplement Funding,39BD001,0719000177S,"IN LOS ANGELES COUNTY, IN EACH DIRECTION ON LA-39 IN THE UNINCORPORATED AREA EMERGENCY PROJECT - EMERGENCY OPENING",37,27,Active,"ON STATE ROUTE: 39. IN LOS ANGELES COUNTY, IN EACH DIRECTION ON LA-39 IN THE UNINCORPORATED AREA EMERGENCY PROJECT - EMERGENCY OPENING",6,4R - Restoration & Rehabilitation,500000.00,61000.00,S ER NONE,NaN,California,7.0,Los Angeles County,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2178,44993,ER01,Emergency Supplement Funding,31TV004,0218000119S,IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKM,63,1,Active,ON STATE ROUTE: 70. IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKMENT,6,4R - Restoration & Rehabilitation,21004300.00,18412317.60,S NON-MPO,NaN,California,2.0,Plumas County,NaN,NaN
2179,44993,ER01,Emergency Supplement Funding,31TV004,0218000119S,IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKM,63,1,Active,ON STATE ROUTE: 70. IN PLUMAS COUNTY AT VARIOUS LOCATIONS FROM BUTTE COUNTY LINE TO 3.1 MILES WEST OF ROUTE 89 REPAIR ROCK SLOPE PROTECTION AND STABILIZE ROADWAY EMBANKMENT,17,Construction Engineering,2686000.00,2377900.00,S NON-MPO,NaN,California,2.0,Plumas County,NaN,NaN
2180,44993,ER01,Em

In [52]:
## move to utils,
# def add_new_description_col(df):
#     df["obligations_amount_string"] = df["obligations_amount"].astype(str)
    
#     df["new_description_col"] = df["program_code_description"] + " for $" + df["obligations_amount_string"]
    
#     df.drop(columns =['obligations_amount_string'], axis=1, inplace=True)
    
#     return df

In [53]:
# (add_new_description_col(df))>>filter(_.program_code_description.notnull())

In [54]:
df.program_code_description.value_counts()

Emergency Supplement Funding                                          590
National Highway Performance Program (NHPP)                           493
Surface Transportation Block Grant                                    386
Highway Safety Improvement Program (HSIP)                             377
Congestion Mitigation & Air Quality Improvement                       104
Transportation Alternatives                                            48
Projects to Reduce PM 2.5 Emissions                                    42
Section 164 Penalties - Use for HSIP Activities                        37
Bridge Formula Program                                                 33
Metropolitan Planning Program                                          19
Safe and Accessible Transportation Options - Metropolitan Planning     18
NHPP Exempt                                                            12
Railway-Highway Crossings Program                                       7
National Highway Freight Program (NHFP

## Test & Export

In [55]:
# GCS_FILE_PATH  = 'gs://calitp-analytics-data/data-analyses/dla/dla-iija'

In [56]:
###test agg. PASS df with title column, concat and 552 rows
##throwingerror
agg = _script_utils.get_clean_data(df, full_or_agg = 'agg')

In [57]:
agg.sample(3)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new,new_description_col
334,44733,2801153,California,S MTC,Y001,National Highway Performance Program (NHPP),0416000024S,6|17|42,4R - Restoration & Rehabilitation|Construction Engineering|Training,"SANTA CLARA COUNTY IN AND NEAR SAN JOSE FROM S280-N101/101 SEPARATION TO ROUTE 17/280 SEPERATION MINOR CONCRETE, MGS RAILING, HMA AND HMA DIKE.",3864500.0,18|19,4.0,85,Santa Clara County,Santa Clara County,nan,None,None,Road Restoration & Rehabilitation in Santa Clara County,"This project is part of the National Highway Performance Program (NHPP) Program, and recieved $3864500.0. This project will Road Restoration & Rehabilitation in Santa Clara County."
1066,44979,5026063,San Buenaventura,L5026SCAG,Y301,Transportation Alternatives,0719000155L,17|28,Construction Engineering|Facilities for Pedestrians and Bicycles,"HARMON BARRANCA PATH & TELEPHONE ROAD, HARMON BARRANCA PATH & RALSTON STREET, HARMON BARRANCA PATH NORTH LINK & ANTELOPE AVE. CONSTRUCT CLASS III BIK",432000.0,26,7.0,111,Ventura County,Ventura County,5026.0,Ventura County Transportation Commission,Southern California Association Of Governments,Facilities for Pedestrians and Bicycles in San Buenaventura,"This project is part of the Transportation Alternatives Program, and recieved $432000.0. This project will Facilities for Pedestrians and Bicycles in San Buenaventura."
586,44816,P035012,California,S MTC,YS30,Highway Safety Improvement Program (HSIP),0419000296S,17|21,Construction Engineering|Safety,"SANTA CLARA AND SAN MATEO COUNTIES IN AND NEAR PORTOLA VALLEY AND WOODSIDE FROM ROUTE 9 TO ROUTE 84 INSTALL CURVE WARNING SIGNS, STRIPING AND PAVEMEN",2349000.0,18,4.0|5.0,81|85|87,San Mateo County|Santa Clara County|Santa Cruz County,San Mateo County & Santa Clara County & Santa Cruz County,nan,None,None,Install Signage in San Mateo County & Santa Clara County & Santa Cruz County,"This project is part of the Highway Safety Improvement Program (HSIP) Program, and recieved $2349000.0. This project will Install Signage in San Mateo County & Santa Clara County & Santa Cruz County."


In [62]:
(agg>>filter(_.county_name_title=="Statewide")).sample()

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new,new_description_col
692,44826,0010209,California,S SANDAG,Y550,State Planning and Research,0022000354S,18,Planning,OWP 22/23 SAN DIEGO REGION CONSOLIDATED PLANNING GRANT (CPG) PROJECT FOR SAN DIEGO ASSOCIATION OF GOVERNMENTS (SANDAG) OVERALL WORK PROGRAM (OWP) FOR,400500.0,0,nan,999,Statewide,Statewide,nan,None,None,Planning and Research in San Diego Association Of Governments,"This project is part of the State Planning and Research Program, and recieved $400500.0. This project will Planning and Research in San Diego Association Of Governments."


In [65]:
(agg>>filter(_.county_name_title=="Statewide")>>filter(_.project_title_new.str.contains("in")))

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new,new_description_col
19,44587,39AZ001,California,S ER NONE,ER01,Emergency Supplement Funding,0119000058S,6|17,4R - Restoration & Rehabilitation|Construction Engineering,IN DEL NORTE COUNTY AT VARIOUS LOCATIONS REPAIR ROADWAY AND CULVERT,2638100.00,0,nan,999,Statewide,Statewide,nan,None,None,Road Restoration & Rehabilitation in Del Norte County,"This project is part of the Emergency Supplement Funding Program, and recieved $2638100.0. This project will Road Restoration & Rehabilitation in Del Norte County."
163,44676,000C533,California,S MTC,Y001,National Highway Performance Program (NHPP),0417000023S,6|17,4R - Restoration & Rehabilitation|Construction Engineering,IN ALAMEDA AND SOLANO COUNTIES AT VARIOUS LOCATION INSTALL HIGHWAY PLANTING AND IRRIGATION SYSTEMS,2224000.00,0,nan,999,Statewide,Statewide,nan,None,None,Install Planting and Irrigation Systems in Alameda,"This project is part of the National Highway Performance Program (NHPP) Program, and recieved $2224000.0. This project will Install Planting and Irrigation Systems in Alameda."
194,44686,000C524,California,S MTC,YS32,Section 164 Penalties - Use for HSIP Activities,0414000003S,17|21|42,Construction Engineering|Safety|Training,ALAMEDA AND CONTRA COSTA COUNTIES AT VARIOUS LOCATIONS CROSSWALK SAFETY ENHANCEMENT,768800.00,0,nan,999,Statewide,Statewide,nan,None,None,Safety Improvements in Alameda,"This project is part of the Section 164 Penalties - Use for HSIP Activities Program, and recieved $768800.0. This project will Safety Improvements in Alameda."
195,44686,000C536,California,S MTC,Y001,National Highway Performance Program (NHPP),0416000036S,6|17,4R - Restoration & Rehabilitation|Construction Engineering,SANTA CLARA AND ALAMEDA COUNTIES ON ROUTE 880 AND 80 AT VARIOUS LOCATIONS FROM ROUTE 880/237 CONNECTOR SEPARATION TO 98TH AVE OVER CROSSING AND ON RO,3070900.00,0,nan,999,Statewide,Statewide,nan,None,None,Road Restoration & Rehabilitation in Santa Clara,"This project is part of the National Highway Performance Program (NHPP) Program, and recieved $3070900.0. This project will Road Restoration & Rehabilitation in Santa Clara."
238,44691,X019032,California,S FCOG,Y001,National Highway Performance Program (NHPP),0616000234S,6|17|42,4R - Restoration & Rehabilitation|Construction Engineering|Training,"FRESNO COUNTY IN FRESNO AND CLOVIS ON ROUTES 41, 99, 168 AND 180 AT VARIOUS LOCATIONS REPLACE AND UPGRADE TRAFFIC MANAGEMENT SYSTEM AND INSTALL FIBER",12798200.00,0,nan,999,Statewide,Statewide,nan,None,None,Road Restoration & Rehabilitation in Fresno County,"This project is part of the National Highway Performance Program (NHPP) Program, and recieved $12798200.0. This project will Road Restoration & Rehabilitation in Fresno County."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
877,44910,000C518,California,S SJCOG,YS30,Highway Safety Improvement Program (HSIP),1016000021S,17,Construction Engineering,"MERCED, SAN JOAQUIN AND STANISLAUS COUNTIES AT VARIOUS LOCATIONS UPGRADE EXISTING GUARDRAIL AND HMA DIKES",22417.58,0,nan,999,Statewide,Statewide,nan,None,None,Upgrade Guardrails in San Joaquin,"This project is part of the Highway Safety Improvement Program (HSIP) Program, and recieved $22417.58. This project will Upgrade Guardrails in San Joaquin."
894,44916,37Y6004,California,S BCAG,ER01,Emergency Supplement Funding,0320000006S,6,4R - Restoration & Rehabilitation,"BUTTE COUNTY ON ROUTE 70 AT VARIOUS LOCATIONS IMPORT BORROW, JACK WELDED AND CORRUGATED STEEL PIPE",1482990.82,0,nan,999,Statewide,Statewide,nan,None,None,Road Restoration & Rehabilitation in Butte 

## Program Code deep dive

In [ ]:
(agg>>filter(_.program_code_description==("nan"))).program_code.value_counts()

In [ ]:
(agg>>filter(_.program_code_description==("nan")))

### Read in Old Codes


(currently used in script)

In [ ]:
#check project codes list
codes = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/FY21-22ProgramCodesAsOf5-25-2022.v2.xlsx"))


In [ ]:
len(codes)

In [ ]:
codes.head()

### New Codes

In [ ]:
## read in new project codes:
new_codes = to_snakecase(pd.read_excel(f"{GCS_FILE_PATH}/Copy of lst_IIJA_Code_20230510.xlsx"))

In [ ]:
new_codes

#### Remove | from codes

In [ ]:
# new_codes['iija_code_2'] = new_codes['iija_code'].str.split("|").str[0]
new_codes['iija_code_2'] = new_codes['iija_code'].str.replace("|","")

In [ ]:
new_codes.sample(5)

#### Codes with ranges

In [ ]:
new_codes>>filter(_.iija_program_code.str.contains("-"))

In [ ]:
# ## checking with the proj
# proj>>filter(_.program_code.str.contains("YP"))

In [ ]:
# proj.program_code.value_counts()

#### Null codes

In [ ]:
len(new_codes>>filter(_.iija_code.isna()))

In [ ]:
null_program_codes = new_codes>>filter(_.iija_code.isna())

In [ ]:
#download and send to contact! 
# null_program_codes.to_csv("lst_IIJA_Code_20230501_null.csv")

In [ ]:
# null_program_codes

### Add acronym code for IIJA programs
https://stackoverflow.com/questions/4355201/creating-acronyms-in-python

In [ ]:
# (proj>>filter(_.program_code==("ER01"), _.program_code_description.notnull())).sample()

In [ ]:
# (proj>>filter(_.program_code==("ER01"), _.program_code_description.notnull())).sample()

In [ ]:
df.head()

##### Check titles
moved to `_script_utils`

In [ ]:
# sorted(list(agg.project_title_new.unique()))

In [ ]:
# agg>>filter(_.implementing_agency.str.contains("California"))

In [ ]:
# agg>>filter(_.project_title_new.str.contains("Caltrans"))

In [ ]:
# len(agg)

In [ ]:
# agg = utils.title_column_names(agg)

In [ ]:
# agg.to_csv(f"{GCS_FILE_PATH}/IIJA_FMIS_AllProject_20230315_ToDLA_agg.csv")

In [ ]:
###test full. PASS title, no concat and 1241 rows
# full = utils.get_clean_data(df, full_or_agg = 'full')

In [ ]:
# full = utils.title_column_names(full)

In [ ]:
# full.to_csv(f"{GCS_FILE_PATH}/FMIS_Projects_Universe_IIJA_Reporting_3-9-23_all.csv")

In [ ]:
# agg = to_snakecase(agg)

In [ ]:
# test_df = agg>>filter(_.implementing_agency.str.contains('Unknown')
                      # | _.implementing_agency.str.contains("Caltrans")
                     # )

In [ ]:
# test_df.sample()

In [ ]:
# locodes = to_snakecase(pd.read_excel(f"gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/locodes_updated7122021.xlsx"))

In [ ]:
# locodes['agency_name'] = locodes['agency_name'].str.upper()

In [ ]:
# locode_names = sorted(list(locodes['agency_name'].unique()), reverse=True)

In [ ]:
# locode_names.remove('ROSS')

In [ ]:
# locode_names.append("State Parks")

In [ ]:
# locode_names

In [ ]:
# pattern = '|'.join(locode_names)

In [ ]:
# test_df.project_title.str.contains(pattern)

In [ ]:
### https://stackoverflow.com/questions/68869434/create-an-pandas-column-if-a-string-from-a-list-matches-from-another-column
#test_df["agencies_found"] = test_df["project_title"].apply(lambda x: next((a for a in locode_names if a in x), ""))
# test_df["agencies_found2"] = test_df["project_title"].map(lambda s: next((name for name in locode_names if name in s), np.nan))

In [ ]:
# test_df["agencies_found2"] = test_df["agencies_found2"].str.title()

In [ ]:
# test_df

In [ ]:
# agg>>filter(_["Project Title New"].str.contains('Unknown'))

In [ ]:
# agg>>filter(_["implementing_agency"].str.contains('Unknown'))

## Next

#### Statewide 

In [66]:
len(agg>>filter(_.project_title_new.str.contains("Statewide")))

9

In [67]:
state = agg>>filter(_.project_title_new.str.contains("Statewide"))

In [68]:
state.sample(5)

,fmis_transaction_date,project_number,implementing_agency,summary_recipient_defined_text_field_1_value,program_code,program_code_description,recipient_project_number,improvement_type,improvement_type_description,project_title,obligations_amount,congressional_district,district,county_code,county_name,county_name_title,implementing_agency_locode,rtpa_name,mpo_name,project_title_new,new_description_col
843,44897,0010046,California,S NON-MPO,Y550,State Planning and Research,7400010046S,18,Planning,"STATEWIDE STATE PLANNING AND RESEARCH PROGRAM, PART I FY 22/23",62715474.0,0,nan,999,Statewide,Statewide,nan,None,None,Planning and Research Statewide,"This project is part of the State Planning and Research Program, and recieved $62715474.0. This project will Planning and Research Statewide."
311,44728,0010211,California,S NON-MPO,Y560,"Research, Development, and Technology Transfer (RD&T)",6500010147S,19,Research,"STATEWIDE STATE PLANNING AND RESEARCH PROGRAM, PART II FY 22/23",14493165.0,0,nan,999,Statewide,Statewide,nan,None,None,Planning and Research Statewide,"This project is part of the Research, Development, and Technology Transfer (RD&T) Program, and recieved $14493165.0. This project will Planning and Research Statewide."
538,44811,JT22001,California,S NON-MPO,Y490,On the Job Skills Training,0088220010S,42,Training,STATEWIDE OJT/SS OJT/SS FY-2022,517791.0,0,nan,999,Statewide,Statewide,nan,None,None,Project Statewide,"This project is part of the On the Job Skills Training Program, and recieved $517791.0. This project will Project Statewide."
710,44830,00PE023,California,S NON-MPO,Y240,Surface Transportation Block Grant,00000PE023S,15,Preliminary Engineering,STATEWIDE STATEWIDE PRELIMINARY ENGINEERING 2022-23,58809712.0,0,nan,999,Statewide,Statewide,nan,None,None,Preliminary Engineering Projects Statewide,"This project is part of the Surface Transportation Block Grant Program, and recieved $58809712.0. This project will Preliminary Engineering Projects Statewide."
711,44830,00PE023,California,S NON-MPO,YS30,Highway Safety Improvement Program (HSIP),00000PE023S,15,Preliminary Engineering,STATEWIDE STATEWIDE PRELIMINARY ENGINEERING 2022-23,41635714.0,0,nan,999,Statewide,Statewide,nan,None,None,Preliminary Engineering Projects Statewide,"This project is part of the Highway Safety Improvement Program (HSIP) Program, and recieved $41635714.0. This project will Preliminary Engineering Projects Statewide."


In [69]:
# #put into function

In [71]:
# locode_names = read_in_locodes()

In [72]:
# locode_names

In [73]:
#test

In [74]:
# state2 = find_alternative_name(state, 'project_title', "name_in_title")

In [75]:
# state2.sample(5)

In [76]:
# state2.sample()

In [77]:
# state2>>select(_.summary_recipient_defined_text_field_1_value, _.implementing_agency, _.program_code_description, _.improvement_type_description,
#                                                       _.project_title, _.obligations_amount, _.county_name, _.county_name_title,
#                                                       _.project_title_new, _.new_description_col, _.name_in_title)

In [78]:
# state2>>filter(_.name_in_title==(""))>>select(_.summary_recipient_defined_text_field_1_value, _.implementing_agency, _.program_code_description, _.improvement_type_description,
#                                                       _.project_title, _.obligations_amount, _.county_name, _.county_name_title,
#                                                       _.project_title_new, _.new_description_col, _.name_in_title)

In [79]:
# state3 = ((find_alternative_name(
#         (state2>>filter(_.name_in_title=="")), 'summary_recipient_defined_text_field_1_value', 'name_in_recp'))
#         >>select(_.summary_recipient_defined_text_field_1_value, _.implementing_agency, _.program_code_description, _.improvement_type_description,
#                                                       _.project_title, _.obligations_amount, _.county_name, _.county_name_title,
#                                                       _.project_title_new, _.new_description_col, _.name_in_title, _.name_in_recp)
# )

In [80]:
# state3